## Anonymizing FasTrak Account Data

In this process of anonymizing the Fastrak account data, we are going to use the [Faker](https://github.com/joke2k/faker) python package. 

In [15]:
import recordlinkage
import pandas as pd
import numpy as np
import unicodecsv as csv
from faker import Factory
from faker import Faker
from collections import defaultdict
import csv 
import re
import fuzzyset
from sklearn.preprocessing import LabelEncoder

In [5]:
compare_cl = recordlinkage.Compare()

In [82]:
clipper = pd.read_csv("~/Box/Data/fake_clipper.csv", encoding = 'utf-8')
fastrak = pd.read_csv("~/Box/Data/fake_fastrak.csv", encoding = 'utf-8')

In [7]:
fastrak

,tracking,fname,lname,fastrak_id,phone,address1,address2,city,state
0,Beyonce Knowles,Beyonce,Knowles,fj28hf83,2024678907,101 Howard St,NaN,San Francisco,California
1,Kanye W,Kanye,W,ofjh983whf983,413-764-9463,55 Polk St,#22,NaN,California
2,Bill Joel,Bill,Joel,fo28h3d8w3,7034779079,493 Haight St,APT 493,San Francisco,California
3,Abigail Cash,Abigail,Cash,dhowijed093,7034779022,1390 Market,NaN,San Francisco,California
4,Abigail Jones,Abigail,Jones,lojsdfo45,918-264-1412,3051 San Bruno Av,NaN,SF,California
5,Chicago West,Chicago,West,ofjh983whf983,413-764-9463,56 Polk St,#23,San Francisco,California
6,Jon Cash,Jon,Cash,oodjfoij,NaN,NaN,NaN,NaN,NaN


In [5]:
#fastrak= fastrak.dropna(subset=['address2', 'city'], how='all')

#fastrak

In [6]:
#fastrak = fastrak[~(fastrak.fastrak_id.str.len() > 10)]
#fastrak

The code bellow, uses some very helpful functions found in a blog post named [A Practical Guide to Anonymizing Datasets with Python & Faker](http://blog.districtdatalabs.com/a-practical-guide-to-anonymizing-datasets-with-python-faker)

In [7]:
# Load the faker and its providers
fake  = Factory.create()
    
# Create empty dictionaries
fnamesdict  = defaultdict(fake.first_name)
lnamesdict  = defaultdict(fake.last_name)
phonedict  = defaultdict(fake.phone_number)
addressdict  = defaultdict(fake.address)

def anonymize_rows(rows):
    """
    Rows is an iterable of dictionaries that contain name and
    email fields that need to be anonymized.
    """

    # Iterate over the rows and yield anonymized rows.
    for row in rows:
        # Replace the name and email fields with faked fields.
        row['fname']  = fnamesdict[row['fname']]
        row['lname']  = lnamesdict[row['lname']]
        row['phone']  = phonedict[row['phone']]
        row['address1']  = addressdict[row['address1']]

        # Yield the row back to the caller
        yield row


def anonymize(source, target):
    """
    The source argument is a path to a CSV file containing data to anonymize,
    while target is a path to write the anonymized CSV data to.
    """
    
    with open(source, 'rU') as f:
        with open(target, 'w') as o:
            # Use the DictReader to easily extract fields
            reader = csv.DictReader(f)
            writer = csv.DictWriter(o, reader.fieldnames)
            # Read and anonymize data, writing to target file.
            for row in anonymize_rows(reader):
                writer.writerow(row)

In [8]:
def anonymize2(source, target):
    """
    The source argument is a path to a CSV file containing data to anonymize,
    while target is a path to write the anonymized CSV data to.
    """
    reader = source.to_dict('records')
    for row in anonymize_rows(reader):
        print row
        print "/n"

In [10]:
ftdict = fastrak.to_dict('records')

In [ ]:
pd.DataFrame(ftdict.items())

In [11]:
ftdict

[{u'address1': u'101 Howard St',
  u'address2': nan,
  u'city': nan,
  u'fastrak_id': u'fj28hf83',
  u'fname': u'Beyonce',
  u'lname': u'Knowles',
  u'phone': u'2024678907',
  u'state': u'California',
  u'tracking': u'Beyonce Knowles'},
 {u'address1': u'55 Polk St',
  u'address2': u'#22',
  u'city': nan,
  u'fastrak_id': u'ofjh983whf983',
  u'fname': u'Kanye',
  u'lname': u'W',
  u'phone': u'413-764-9463',
  u'state': u'California',
  u'tracking': u'Kanye W'},
 {u'address1': u'493 Haight St',
  u'address2': u'APT 493',
  u'city': u'San Francisco',
  u'fastrak_id': u'fo28h3d8w3',
  u'fname': u'Bill',
  u'lname': u'Joel',
  u'phone': u'7034779079',
  u'state': u'California',
  u'tracking': u'Bill Joel'},
 {u'address1': u'1390 Market',
  u'address2': nan,
  u'city': u'San Francisco',
  u'fastrak_id': u'dhowijed093',
  u'fname': u'Abigail',
  u'lname': u'Cash',
  u'phone': u'7034779022',
  u'state': u'California',
  u'tracking': u'Abigail Cash'},
 {u'address1': u'3051 San Bruno Av',
  u'ad

In [12]:
#anonymize("/Users/smcnish/Box/Data/fake_fastrak.csv", "/Users/smcnish/Box/Data/very_fake.csv")

anonymize2(fastrak, "/Users/smcnish/Box/Data/very_fake_ft.csv")

{u'city': nan, u'tracking': u'Beyonce Knowles', u'address1': u'8891 Henry Garden\nJacobsport, FM 58829', u'address2': nan, u'lname': u'West', u'phone': u'02653546417', u'state': u'California', u'fname': u'David', u'fastrak_id': u'fj28hf83'}
/n
{u'city': nan, u'tracking': u'Kanye W', u'address1': u'65046 Clayton Freeway\nLake Dianaville, MA 77832', u'address2': u'#22', u'lname': u'Mason', u'phone': u'01487224138', u'state': u'California', u'fname': u'James', u'fastrak_id': u'ofjh983whf983'}
/n
{u'city': u'San Francisco', u'tracking': u'Bill Joel', u'address1': u'7049 Rice Locks Apt. 838\nDebraberg, CT 40557-6292', u'address2': u'APT 493', u'lname': u'Weaver', u'phone': u'227-565-8956x81715', u'state': u'California', u'fname': u'Bethany', u'fastrak_id': u'fo28h3d8w3'}
/n
{u'city': u'San Francisco', u'tracking': u'Abigail Cash', u'address1': u'60260 Chad Viaduct\nAllisonfort, LA 97526-8893', u'address2': nan, u'lname': u'Garcia', u'phone': u'(602)504-7519x806', u'state': u'California', u'

In [13]:
#anonymize("/Users/smcnish/Box/Data/fake_clipper.csv", "/Users/smcnish/Box/Data/clipper_very_fake.csv")

In [14]:
clipperfake = pd.read_csv("~/Box/Data/clipper_very_fake.csv", encoding = 'utf-8')
fastrakfake = pd.read_csv("~/Box/Data/very_fake.csv", encoding = 'utf-8')

In [15]:
fastrakfake

,Beyonce Knowles,Emily,Green,fj28hf83,02041897075,"63953 John Island Suite 788 West Timothy, VI 09411",Unnamed: 6,Unnamed: 7,California
0,Kanye W,Michael,Jacobs,ofjh983whf983,1-208-012-0723x693,"1032 Butler Lake\nNew Katherine, WI 41944",#22,NaN,California
1,Bill Joel,Stuart,Williams,fo28h3d8w3,846.850.5296x00098,"017 Monroe Port Apt. 840\nEast Christopher, MO...",APT 493,San Francisco,California
2,Abigail Cash,Lisa,Atkins,dhowijed093,1-319-296-6112,"56406 Kim Viaduct\nEast Amyhaven, WA 20577-9513",NaN,San Francisco,California
3,Abigail Jones,Lisa,Ryan,lojsdfo45,(437)022-4410x10229,"19553 Schmidt Motorway\nKimberlymouth, MT 6385...",NaN,SF,California
4,Chicago West,Kelly,Lyons,ofjh983whf983,1-208-012-0723x693,"73596 James Well Suite 519\nScottport, HI 6924...",#23,San Francisco,California


In [16]:
clipperfake

,Beyonce Knowles,Timothy,Patterson,1928475,224-746-5578,"53777 Santiago Ville Williamsmouth, UT 49847-2053",Unnamed: 6,San Francisco,California
0,Billy Joel,Desiree,King,923847,846.698.2778,"3083 Gregory Gateway\nPort Davidstad, VA 11799...",#493,San Francisco,CA
1,Jon Cash,Brittany,Schultz,812640127,02923774097,"0201 Jasmin Skyway\nSouth Jennifer, WV 24421",NaN,San Francisco,California
2,Abby Jones,Daniel,Wiley,9274917,217-462-0136x02994,"783 Navarro Gardens Apt. 243\nPort Thomasside,...",NaN,San Francisco,NaN
3,Kanye West,Tanya,James,45626774,140.385.8238x188,"551 Bond Garden\nNorth Denise, NH 12222-0665",NaN,San Francisco,California


In [17]:
print fnamesdict['Beyonce']

David


In [ ]:
from faker import Faker
fake = Faker()
print("".join(fake.name().split(" ")))

### Label encoding to anonymize data testing

In [37]:
clipper

,tracking,fname,lname,clipper_id,phone,address1,address2,city,state
0,Beyonce Knowles,Beyonce,Knowles,1928475,2024678907,101 Howard St,NaN,San Francisco,California
1,Billy Joel,Billy,Joel,923847,+1 703 477 9079,493 Haight St,#493,San Francisco,CA
2,Jon Cash,Jon,Cash,812640127,2136630484,1390 Market Street,NaN,San Francisco,California
3,Abby Jones,Abby,Jones,9274917,918-264-1412,3051 San Bruno Av,NaN,San Francisco,NaN
4,Kanye West,Kanye,West,45626774,(413) 764 - 9463,55 Polk,NaN,San Francisco,California
5,Jon Cash,Jon,cash,812640127,2136630484,1390 market street,NaN,san francisco,california


In [38]:
fastrak

,tracking,fname,lname,fastrak_id,phone,address1,address2,city,state
0,Beyonce Knowles,Beyonce,Knowles,fj28hf83,2024678907,101 Howard St,NaN,San Francisco,California
1,Kanye W,Kanye,W,ofjh983whf983,413-764-9463,55 Polk St,#22,NaN,California
2,Bill Joel,Bill,Joel,fo28h3d8w3,7034779079,493 Haight St,APT 493,San Francisco,California
3,Abigail Cash,Abigail,Cash,dhowijed093,7034779022,1390 Market,NaN,San Francisco,California
4,Abigail Jones,Abigail,Jones,lojsdfo45,918-264-1412,3051 San Bruno Av,NaN,SF,California
5,Chicago West,Chicago,West,ofjh983whf983,413-764-9463,56 Polk St,#23,San Francisco,California
6,Jon Cash,Jon,Cash,oodjfoij,NaN,NaN,NaN,NaN,NaN


In [73]:
def process_dfs(fastrak1, clipper1, col):
    
    # crerating a dataframe with all the unique columns (names) to fit the model
    names = pd.DataFrame(clipper1[col].unique(), columns=[col]).append(pd.DataFrame(fastrak1[col].unique(), columns=[col]))
    names1 = pd.DataFrame(names[col].unique(), columns=[col])
    print names1
    
    # fit the model
    le=LabelEncoder()
    le.fit(names1[col].values)
    
    # transform fastrak and clipper
    fastrak1[col]=le.transform(fastrak1[col])
    clipper1[col] = le.transform(clipper1[col])
    return fastrak1, clipper1

In [77]:
fastrak, clipper = process_dfs(fastrak, clipper, "lname")

     lname
0  Knowles
1     Joel
2     Cash
3    Jones
4     West
5     cash
6        W


In [85]:
fastrak["fname"] = fastrak["fname"].str.lower()

In [90]:
a = fastrak.drop(["fname"], axis = 1)
a

,tracking,lname,fastrak_id,phone,address1,address2,city,state
0,Beyonce Knowles,Knowles,fj28hf83,2024678907,101 Howard St,NaN,San Francisco,California
1,Kanye W,W,ofjh983whf983,413-764-9463,55 Polk St,#22,NaN,California
2,Bill Joel,Joel,fo28h3d8w3,7034779079,493 Haight St,APT 493,San Francisco,California
3,Abigail Cash,Cash,dhowijed093,7034779022,1390 Market,NaN,San Francisco,California
4,Abigail Jones,Jones,lojsdfo45,918-264-1412,3051 San Bruno Av,NaN,SF,California
5,Chicago West,West,ofjh983whf983,413-764-9463,56 Polk St,#23,San Francisco,California
6,Jon Cash,Cash,oodjfoij,NaN,NaN,NaN,NaN,NaN


In [91]:
fastrak

,tracking,fname,lname,fastrak_id,phone,address1,address2,city,state
0,Beyonce Knowles,beyonce,Knowles,fj28hf83,2024678907,101 Howard St,NaN,San Francisco,California
1,Kanye W,kanye,W,ofjh983whf983,413-764-9463,55 Polk St,#22,NaN,California
2,Bill Joel,bill,Joel,fo28h3d8w3,7034779079,493 Haight St,APT 493,San Francisco,California
3,Abigail Cash,abigail,Cash,dhowijed093,7034779022,1390 Market,NaN,San Francisco,California
4,Abigail Jones,abigail,Jones,lojsdfo45,918-264-1412,3051 San Bruno Av,NaN,SF,California
5,Chicago West,chicago,West,ofjh983whf983,413-764-9463,56 Polk St,#23,San Francisco,California
6,Jon Cash,jon,Cash,oodjfoij,NaN,NaN,NaN,NaN,NaN


In [84]:
clipper

,tracking,fname,lname,clipper_id,phone,address1,address2,city,state
0,Beyonce Knowles,Beyonce,Knowles,1928475,2024678907,101 Howard St,NaN,San Francisco,California
1,Billy Joel,Billy,Joel,923847,+1 703 477 9079,493 Haight St,#493,San Francisco,CA
2,Jon Cash,Jon,Cash,812640127,2136630484,1390 Market Street,NaN,San Francisco,California
3,Abby Jones,Abby,Jones,9274917,918-264-1412,3051 San Bruno Av,NaN,San Francisco,NaN
4,Kanye West,Kanye,West,45626774,(413) 764 - 9463,55 Polk,NaN,San Francisco,California
5,Jon Cash,Jon,cash,812640127,2136630484,1390 market street,NaN,san francisco,california


In [ ]:
def process_dfs(fastrak, clipper, col):
    
    # crerating a dataframe with all the unique columns (names) to fit the model
    names = pd.DataFrame(clipper[col].unique(), columns=[col]).append(pd.DataFrame(fastrak[col].unique(), columns=[col]))
    names1 = pd.DataFrame(names[col].unique(), columns=[col])
    
    # fit the model
    le=LabelEncoder()
    le.fit(names1[col].values)
    
    # transform fastrak and clipper
    fastrak[col]=le.transform(fastrak[col])
    clipper[col] = le.transform(clipper[col])
    return fastrak, clipper

We need to have the same names for the columns! So first think we need to do it to make sure First Name, Last Name, State, City, Zip Code

Also we can drop all the columns that are not going to be used in the anonymization process. 

FasTrak columns:

Account,Account_Type,Account_Status,Account_Opening_Data,City,STATE,Device,Device Status,FN_ANONYMOUS,LN_Anonymous

Clipper columns:

CONTACT_ID,CITY,STATE_,CARD_SERIAL_NUMBER,CARD_TYPE,STATUS,FN_Anonymous,LN_Anonymous

In [45]:
fastrak

,track,first_name,lname,fastrak_id,phone,address1,address2,city,state
0,Beyonce Knowles,2,3,fj28hf83,2024678907,101 Howard St,NaN,San Francisco,California
1,Kanye W,7,4,ofjh983whf983,413-764-9463,55 Polk St,#22,NaN,California
2,Bill Joel,3,1,fo28h3d8w3,7034779079,493 Haight St,APT 493,San Francisco,California
3,Abigail Cash,1,0,dhowijed093,7034779022,1390 Market,NaN,San Francisco,California
4,Abigail Jones,1,2,lojsdfo45,918-264-1412,3051 San Bruno Av,NaN,SF,California
5,Chicago West,5,5,ofjh983whf983,413-764-9463,56 Polk St,#23,San Francisco,California
6,Jon Cash,6,0,oodjfoij,NaN,NaN,NaN,NaN,NaN


In [43]:
# renaming the columns for homogeneity between the two datesets
fastrak.rename(columns={ ' Last_Name ' : 'last_name',
                         ' First_Name ' : 'first_name',
                         ' STATE ' : 'state',
                         ' City ' : 'city',
                         ' ZIPCODE ' : 'zip_code',
                         '' : ''}
               , inplace=True)

cliper.rename(columns={ '' : 'last_name',
                         '' : 'first_name',
                         '' : 'state',
                         '' : 'city',
                         '' : 'zip_code',
                         '' : ''}
               , inplace=True)

# dropping the columns that are not going to be needed
fastrak.drop(['Account_Type', 'Account_Status', 'Account_Opening_Data', 
              'Device', 'Device Status', ''], axis=1)

clipper.drop(['CONTACTID', 'CARD_TYPE', 'STATUS', 
              '', '', ''], axis=1)

We also need to lower case all the names

In [ ]:
# lowercasing: first name, last name, state, city.  

fastrak["last_name"] = fastrak["last_name"].str.lower()
fastrak["first_name"] = fastrak["first_name"].str.lower()
fastrak["state"] = fastrak["state"].str.lower()
fastrak["city"] = fastrak["city"].str.lower()

clipper["last_name"] = clipper["last_name"].str.lower()
clipper["first_name"] = clipper["first_name"].str.lower()
clipper["state"] = clipper["state"].str.lower()
clipper["city"] = clipper["city"].str.lower()

In [ ]:
def process_dfs(fastrak1, clipper1, col):
    
    # crerating a dataframe with all the unique columns (names) to fit the model
    names = pd.DataFrame(clipper1[col].unique(), columns=[col]).append(pd.DataFrame(fastrak1[col].unique(), columns=[col]))
    names1 = pd.DataFrame(names[col].unique(), columns=[col])
    
    # fit the model
    le=LabelEncoder()
    le.fit(names1[col].values)
    
    # transform fastrak and clipper
    fastrak1[col]=le.transform(fastrak1[col])
    clipper1[col] = le.transform(clipper1[col])
    return fastrak, clipper

In [ ]:
process_